In [3]:
#import common libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy
import os
import csv


In [4]:
#read the csv data, you can change the file name according to yours
raw_data = pd.read_csv(r'C:\Users\user\Desktop\dataset\train.csv', delimiter=',')

In [5]:
#encode categorical features into numbers, for lose_reason, if no result yet, set to 0
cleanup_nums1 = {"type": {"short service": 0, "long service": 1, "clear": 2, "push/rush": 3, "smash": 4, "defensive shot": 5,
                          "drive": 6, "net shot": 7, "lob": 8, "drop": 9}}
cleanup_nums2 = {"lose_reason": {"out": 1, "touched the net": 2, "not pass over the net": 3, "opponent's ball landed": 4, "misjudged": 5}}
data = raw_data.replace(cleanup_nums1)
data = data.replace(cleanup_nums2)
data['lose_reason'].fillna(0, inplace=True)

In [6]:
#these are the features that may be important for prediction, to see what method was used for feature selection, check the description on Inter-Stroke Features Selection
selected = ['match_id', 'set', 'rally_id', 'ball_round', 'rally_length', 
            'landing_height', 'landing_area', 
            'player_location_x', 'player_location_y',
            'opponent_location_x', 'opponent_location_y',
            'landing_x', 'landing_y','type']

In [7]:
#some of the column may need further scaling
selected_data = data[selected]
selected_data.iloc[:,6:-1]

,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y
0,7,170.8,308.95,236.7,675.21,0.773171,0.519687
1,7,208.7,594.37,176.8,318.46,0.086585,-0.237240
2,3,174.2,378.28,180.1,622.54,1.402439,1.638490
3,4,257.9,809.08,179.3,306.43,1.358537,-1.434167
4,1,273.5,268.22,233.2,738.15,1.487805,0.359844
...,...,...,...,...,...,...,...
30167,7,212.5,659.34,172.5,212.66,0.051220,-0.436198
30168,7,162.8,349.63,189.4,647.10,0.145122,0.290938
30169,7,172.4,583.50,175.9,344.84,-0.097561,-0.277240
30170,9,156.7,401.02,180.6,583.43,-0.080488,1.515417


In [55]:
train = selected_data
from sklearn.preprocessing import MinMaxScaler

# Select the columns that need scaling
columns_to_scale = selected_data.columns[5:-1]

# Create an instance of MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))

# Fit and transform the selected columns using the scaler
train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])
train.iloc[:,5:-1]

C:\Users\user\AppData\Local\Temp\ipykernel_22324\3694319556.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[columns_to_scale] = scaler.fit_transform(train[columns_to_scale])


,landing_height,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y
0,1.0,0.333333,-0.061431,-0.476638,0.455415,0.511765,0.214696,0.267701
1,1.0,0.333333,0.233281,0.346836,0.046122,-0.475245,0.018631,-0.118990
2,-1.0,-0.555556,-0.034992,-0.276612,0.068671,0.366045,0.394393,0.839262
3,-1.0,-0.333333,0.615863,0.966302,0.063205,-0.508528,0.381856,-0.730463
4,1.0,-1.000000,0.737170,-0.594149,0.431500,0.685900,0.418771,0.186042
...,...,...,...,...,...,...,...,...
30167,1.0,0.333333,0.262830,0.534283,0.016741,-0.767959,0.008532,-0.220632
30168,1.0,0.333333,-0.123639,-0.359271,0.132217,0.433994,0.035347,0.150839
30169,1.0,0.333333,-0.048989,0.315474,0.039973,-0.402260,-0.033954,-0.139425
30170,-1.0,0.777778,-0.171073,-0.211004,0.072087,0.257840,-0.029079,0.776388


In [9]:
#group the data by the same rally for we need to make sure the sequence our model observe is on the same rally
grouped_data = train.groupby(['set', 'match_id', 'rally_id'])

In [56]:
#generate past information(past 4 strokes) and future information(we need to predict future stroke)
sequence_length = 4
future_length = 1
sequences = []
futures = []
for _, group in grouped_data:
    rally_data = group[[ 'landing_height', 'landing_area', 
            'player_location_x', 'player_location_y',
            'opponent_location_x', 'opponent_location_y',
            'landing_x', 'landing_y']]  # Select the relevant features for the input
    
    num_samples = len(rally_data) - sequence_length + 1 - future_length
    for i in range(num_samples):
        sequence = rally_data.iloc[i:i+sequence_length]
        sequences.append(sequence) 
        future = rally_data.iloc[i+sequence_length:i+sequence_length+1]
        futures.append(future)

In [57]:
sequences[2]

,landing_height,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y
2,-1.0,-0.555556,-0.034992,-0.276612,0.068671,0.366045,0.394393,0.839262
3,-1.0,-0.333333,0.615863,0.966302,0.063205,-0.508528,0.381856,-0.730463
4,1.0,-1.000000,0.737170,-0.594149,0.431500,0.685900,0.418771,0.186042
5,1.0,-0.777778,0.630638,0.286940,0.482064,-0.627080,0.364444,-0.110236


In [58]:
futures[100]

,landing_height,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y
156,1.0,0.333333,0.190513,0.26787,0.178681,-0.482466,0.093853,0.010962


In [2]:
#import libraries for model archeitecture
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
import tensorflow as tf

In [59]:
#convert the dataframes into numpy arrays
X = np.array(sequences)
y = np.array(futures)
X.shape, y.shape

((21100, 4, 8), (21100, 1, 8))

In [60]:
#split the datasets 70% training, 20% testing, 10% validating

from sklearn.model_selection import train_test_split

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X_shuffled = X[indices]
y_shuffled = y[indices]

# Split the shuffled data into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_shuffled, y_shuffled, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.33, random_state=42)

# Verify the shapes of the data splits
print("Train data shape:", X_train.shape)
print("Test data shape:", X_test.shape)
print("Validation data shape:", X_val.shape)
print("Train target shape:", y_train.shape)
print("Test target shape:", y_test.shape)
print("Validation target shape:", y_val.shape)


Train data shape: (14770, 4, 8)
Test data shape: (4241, 4, 8)
Validation data shape: (2089, 4, 8)
Train target shape: (14770, 1, 8)
Test target shape: (4241, 1, 8)
Validation target shape: (2089, 1, 8)


In [124]:
# E1D1
# an easy encoder-decoder system, one encoder and one decoder is build
#

n_past = 4
n_future = 1
n_features = 8

# Define input layer
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))

# Encoder LSTM
encoder_l1 = tf.keras.layers.LSTM(32, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]

decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)


#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 4, 8)]       0           []                               
                                                                                                  
 lstm_32 (LSTM)                 [(None, 32),         5248        ['input_12[0][0]']               
                                 (None, 32),                                                      
                                 (None, 32)]                                                      
                                                                                                  
 repeat_vector_11 (RepeatVector  (None, 1, 32)       0           ['lstm_32[0][0]']                
 )                                                                                         

In [132]:
X_train[0]
batch_size = 32
epochs = 25

In [235]:
X_train.shape

(14770, 4, 8)

In [63]:
X_train[0]

array([[ 1.        , -1.        ,  0.36625194, -0.35309646, -0.26614281,
         0.6600866 ,  0.35155842,  0.21996647],
       [-1.        ,  0.55555556,  0.56687403,  0.35488524,  0.22856167,
        -0.40212204, -0.12728539, -0.68975335],
       [ 1.        ,  0.55555556, -0.30404355, -0.77888374,  0.2046464 ,
         0.51469795,  0.25230716,  0.5619562 ],
       [-1.        ,  0.11111111,  0.23950233,  0.65661199, -0.11923471,
        -0.69411667,  0.28330141, -0.56485645]])

In [130]:
X_train[278,    0,    0] = 1
y_train[2129,    0,    0] = 1
y_train[4630,    0,    0] = 1
y_train[6365,    0,    0] = 1
y_val[1260,    0,    0] = 1

In [140]:
np.argwhere(np.isnan(X_test))

array([], shape=(0, 3), dtype=int64)

In [143]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-6 * 0.90 ** x)
model_e1d1.compile(optimizer= tf.keras.optimizers.Adam(clipnorm=1) , loss= 'huber')
history_e1d1 = model_e1d1.fit(X_train,y_train ,
                         batch_size=batch_size, epochs=epochs,
                         validation_data=(X_val, y_val),verbose=1,callbacks=[reduce_lr])


Epoch 1/25
462/462 [==============================] - 15s 6ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 1.0000e-06
Epoch 2/25
462/462 [==============================] - 2s 5ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 9.0000e-07
Epoch 3/25
462/462 [==============================] - 2s 5ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 8.1000e-07
Epoch 4/25
462/462 [==============================] - 2s 5ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 7.2900e-07
Epoch 5/25
462/462 [==============================] - 2s 4ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 6.5610e-07
Epoch 6/25
462/462 [==============================] - 2s 4ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 5.9049e-07
Epoch 7/25
462/462 [==============================] - 2s 5ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 5.3144e-07
Epoch 8/25
462/462 [==============================] - 2s 5ms/step - loss: 0.0880 - val_loss: 0.0889 - lr: 4.7830e-07
Epoch 9/25
462/462 [==============================] - 2s 5ms/st

In [138]:
X_val.shape

(2089, 4, 8)

In [144]:
test_loss = model_e1d1.evaluate(
    X_test,
    y_test,
    verbose=0
)


In [146]:
predictions = model_e1d1.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)  # Assuming it's a classification problem
true_labels = np.argmax(y_test, axis=1)  # Assuming one-hot encoded true labels

accuracy = np.mean(predicted_labels == true_labels)
print("Test Accuracy:", accuracy)


133/133 [==============================] - 1s 2ms/step
Test Accuracy: 1.0


In [145]:
test_loss

0.0876418948173523

In [149]:
# Save the model
model_e1d1.save('model_e1d1_1.h5')

In [211]:
test_v1_data = pd.read_csv(r'C:\Users\user\Desktop\dataset\val_given.csv', delimiter=',')

In [212]:
first = test_v1_data[:4]
first

,rally,ball_round,time,frame_num,roundscore_A,roundscore_B,player,type,aroundhead,backhand,...,player_location_area,player_location_x,player_location_y,opponent_location_area,opponent_location_x,opponent_location_y,set,match_id,rally_id,rally_length
0,3,1,00:53:53,97017,2,1,11,short service,0.0,1,...,8,180.5,616.63,8,132.8,301.55,3,38,3134,10
1,3,2,00:53:54,97042,2,1,5,lob,0.0,0,...,7,156.6,370.18,8,174.2,624.69,3,38,3134,10
2,3,3,00:53:56,97087,2,1,11,drop,0.0,0,...,9,245.0,785.78,8,174.1,318.30,3,38,3134,10
3,3,4,00:53:56,97108,2,1,5,net shot,0.0,0,...,8,127.6,348.47,8,199.9,733.41,3,38,3134,10


In [213]:
first = first[selected]
first = first.replace(cleanup_nums1)
first = first.replace(cleanup_nums2)
first

,match_id,set,rally_id,ball_round,rally_length,landing_height,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y,type
0,38,3,3134,1,10,2.0,7,180.5,616.63,132.8,301.55,-0.309756,-0.361563,0
1,38,3,3134,2,10,1.0,3,156.6,370.18,174.2,624.69,1.200000,1.576771,8
2,38,3,3134,3,10,2.0,8,245.0,785.78,174.1,318.30,-0.748780,-0.571250,9
3,38,3,3134,4,10,2.0,7,127.6,348.47,199.9,733.41,0.063415,0.464479,7


In [214]:
selected_xy = [ 'landing_height', 'landing_area', 
            'player_location_x', 'player_location_y',
            'opponent_location_x', 'opponent_location_y',
            'landing_x', 'landing_y']
first = first[selected_xy]
test_xy_1 = first
test_xy_t = test_xy_1
from sklearn.preprocessing import MinMaxScaler

# Select the columns that need scaling
columns_to_scale = selected_data.columns[5:-1]


# Fit and transform the selected columns using the scaler
test_xy_t = scaler.fit_transform(test_xy_t[:])
test_xy_t

array([[ 1.        ,  0.6       , -0.0988075 ,  0.2264069 , -1.        ,
        -1.        , -0.5494368 , -0.80476214],
       [-1.        , -1.        , -0.50596252, -0.90071117,  0.23397914,
         0.4965035 ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  1.        ,  1.        ,  0.23099851,
        -0.92242856, -1.        , -1.        ],
       [ 1.        ,  0.6       , -1.        , -1.        ,  1.        ,
         1.        , -0.16645807, -0.03564328]])

In [215]:
test_xy_1

,landing_height,landing_area,player_location_x,player_location_y,opponent_location_x,opponent_location_y,landing_x,landing_y
0,2.0,7,180.5,616.63,132.8,301.55,-0.309756,-0.361563
1,1.0,3,156.6,370.18,174.2,624.69,1.200000,1.576771
2,2.0,8,245.0,785.78,174.1,318.30,-0.748780,-0.571250
3,2.0,7,127.6,348.47,199.9,733.41,0.063415,0.464479


In [216]:
test_xy_t.shape

(4, 8)

In [217]:
scaler.inverse_transform(test_xy_t)

array([[ 2.00000000e+00,  7.00000000e+00,  1.80500000e+02,
         6.16630000e+02,  1.32800000e+02,  3.01550000e+02,
        -3.09756098e-01, -3.61562500e-01],
       [ 1.00000000e+00,  3.00000000e+00,  1.56600000e+02,
         3.70180000e+02,  1.74200000e+02,  6.24690000e+02,
         1.20000000e+00,  1.57677083e+00],
       [ 2.00000000e+00,  8.00000000e+00,  2.45000000e+02,
         7.85780000e+02,  1.74100000e+02,  3.18300000e+02,
        -7.48780488e-01, -5.71250000e-01],
       [ 2.00000000e+00,  7.00000000e+00,  1.27600000e+02,
         3.48470000e+02,  1.99900000e+02,  7.33410000e+02,
         6.34146341e-02,  4.64479167e-01]])

In [218]:
predictions = model_e1d1.predict(test_xy_t.reshape(1,4,8))
predictions

1/1 [==============================] - 0s 16ms/step


array([[[ 0.37803024, -0.05031022, -0.49182022,  0.2967413 ,
         -0.24866445, -0.731838  , -0.17927802, -0.44825616]]],
      dtype=float32)

In [219]:
original_predictions = scaler.inverse_transform(predictions.reshape(1,8))
original_predictions

array([[1.6890152e+00, 5.3742242e+00, 1.5743015e+02, 6.3200897e+02,
        1.5800731e+02, 3.5945422e+02, 5.0923005e-02, 2.1328626e-02]],
      dtype=float32)